# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

- Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

  - La pregunta que será conservada es aquella que fue enviada primero de acuerdo al timestamp
          
- Un archivo con las preguntas depuradas (eliminando las repeticiones)

1. Se importa los modulos necesarios para esta etapa

In [9]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
from utils import save_df

nlp = spacy.load('es_core_news_md')

2. Se importan las preguntas restantes del paso previo

In [10]:
limpio_df = pd.read_csv('./out_datasets/2-4_base_limpia.csv')

3. Se vulve a normalizas las preguntas restantes.

In [11]:
def replace_accented(match):
    accented = 'áéíóúñüÁÉÍÓÚÑÜ'
    unaccented = 'aeiounuAEIOUNU'
    trans_table = str.maketrans(accented, unaccented)
    return match.translate(trans_table)

stop_words = nltk.corpus.stopwords.words('spanish')
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\sáéíóúñüÁÉÍÓÚÑÜ]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [replace_accented(token) for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [12]:
limpio_df['pregunta_norm'] = list(map(normalize_document, limpio_df['pregunta']))
limpio_df['pregunta_norm'] = limpio_df['pregunta_norm'].astype(str)

In [13]:
display(limpio_df)

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,pregunta_norm
0,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,mexico muchas personas quejan sistema salud co...
1,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,estrategias planes concretos proponen asegurar...
2,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00,dentro plan gobierno acciones dirigidas crear ...
3,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,sigue corrupcion pesar saber donde si origuina
4,Guanajuato,40,Masculino,No Aplica,Transparencia,Quisiera participar en estás elecciones,2024-03-21 23:58:00,quisiera participar elecciones
...,...,...,...,...,...,...,...,...
15422,Ciudad de México,21,Masculino,No Aplica,Educación,¿En algunos sectores de México hay más univers...,2024-02-21 09:19:00,sectores mexico universidades privadas publica...
15423,Ciudad de México,57,Masculino,No Aplica,Violencia en contra de las mujeres,Que dependencia u organismo se hará responsabl...,2024-02-20 20:11:00,dependencia u organismo hara responsable acomp...
15424,Baja California,68,Masculino,No Aplica,Salud,Qué porcentaje del Presupuesto de Egresos de l...,2024-02-20 20:06:00,porcentaje presupuesto egresos federacion dest...
15425,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,hara poder favorecer mexicanos vulnerables alg...


4. Se procesan las pregunas ya normalizadas y se calcula la similitud entre las preguntas

In [14]:
preguntas = limpio_df['pregunta_norm']

vectores = [nlp(str(pregunta)).vector for pregunta in preguntas]

similitud = cosine_similarity(vectores)

5. Ya con la similitud calculada se comparan todas contra todas y se elimina la pregunta mas reciente y parecida.

In [15]:
repetidas = []

for index, fila in limpio_df.iterrows():
    for j in range(index + 1, len(preguntas)):
        if similitud[index, j] > 0.9:
            p1_fecha = limpio_df.loc[index]['fecha']
            p2_fecha = limpio_df.loc[j]['fecha']
            if p1_fecha < p2_fecha:
              repetidas.append(pd.DataFrame([[limpio_df.iloc[j]['pregunta'], limpio_df.loc[index]['pregunta']]], index=None))
            else:
              repetidas.append(pd.DataFrame([[limpio_df.loc[index]['pregunta'], limpio_df.iloc[j]['pregunta']]], index=None))
repetidas = pd.concat(repetidas)
repetidas.columns = ['repetida_de', 'pregunta']

In [16]:
display(repetidas)

,repetida_de,pregunta
0,En México muchas personas se quejan del sistem...,Cómo podría usted cambiar el sistema de salud ...
0,En México muchas personas se quejan del sistem...,¿Cómo podrías incrementar la eficacia de los ...
0,En México muchas personas se quejan del sistem...,Que nos digan su opinión con respecto al servi...
0,En México muchas personas se quejan del sistem...,El sistema de salud es insuficiente mi pregunt...
0,En México muchas personas se quejan del sistem...,¿Cuál es su plan concreto para mejorar el acce...
...,...,...
0,¿Cuál sería su enfoque para abordar la violenc...,¿Cuál es su plan para abordar de manera efecti...
0,¿Cómo abordará el problema de la desigualdad e...,¿Cómo abordará la brecha educativa entre zonas...
0,¿Cuál es su enfoque para abordar la violencia ...,¿Cuál es la estrategia proponen para prevenir ...
0,"¿Cuáles son sus propuestas para concientizar, ...",¿Qué estrategias implementarán para sensibiliz...


6. Por ultimo se guarda el DF de las palabras repetidas

In [17]:
save_df(repetidas, "3_preguntas_repetidas.csv", index=False)

In [18]:
len(set(repetidas['repetida_de']))

6161

7. Se guarda el DF ya completamente limpio

In [19]:

limpio_df = limpio_df[limpio_df['pregunta'].isin(repetidas['pregunta']) == False]
limpio_df = limpio_df.drop(columns=['pregunta_norm'])
display(limpio_df)

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha
0,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00
1,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00
3,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00
4,Guanajuato,40,Masculino,No Aplica,Transparencia,Quisiera participar en estás elecciones,2024-03-21 23:58:00
5,México,14,Femenino,Personas de la diversidad sexual,Transparencia,¿Cuál es su iniciativa para disminuir la censu...,2024-03-21 23:58:00
...,...,...,...,...,...,...,...
15422,Ciudad de México,21,Masculino,No Aplica,Educación,¿En algunos sectores de México hay más univers...,2024-02-21 09:19:00
15423,Ciudad de México,57,Masculino,No Aplica,Violencia en contra de las mujeres,Que dependencia u organismo se hará responsabl...,2024-02-20 20:11:00
15424,Baja California,68,Masculino,No Aplica,Salud,Qué porcentaje del Presupuesto de Egresos de l...,2024-02-20 20:06:00
15425,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00


In [20]:
save_df(limpio_df, "3-1_base_limpia.csv", index=False)